In [ ]:
# 57.26006, 12.11076
meteo_path = f'/lustre/storeB/immutable/archive/projects/metproduction/MEPS/2023/01/01/meps_det_2_5km_20230101T00Z.nc'
meteo_data = xr.open_dataset(meteo_path, engine="netcdf4")
snap_data = xr.open_dataset('output_files_t6/ringhals_20230130_00Z.nc')

In [ ]:
def convert_coords(model, old_long, old_lat):
    pr = model["projection"]

    if pr.grid_mapping_name == 'rotated_latitude_longitude':

        projection = ccrs.RotatedPole(
        pole_longitude=pr.attrs["grid_north_pole_longitude"],
        pole_latitude=pr.attrs["grid_north_pole_latitude"],)

        crs = CRS.from_wkt(projection.to_wkt())

        # Transform from lat-lon (ETEX data) to rotated pole coordinate system (harmonie data)
        transformer = Transformer.from_crs("+proj=longlat +lon_wrap=180", crs)

        new_lon, new_lat = transformer.transform(old_long, old_lat)

    elif pr.grid_mapping_name == 'latitude_longitude':
        
        new_lon, new_lat = old_long, old_lat

    elif pr.grid_mapping_name == 'lambert_conformal_conic':
            projection = ccrs.LambertConformal(
                    central_longitude=pr.attrs["longitude_of_central_meridian"],
                    central_latitude=pr.attrs["latitude_of_projection_origin"],
                    standard_parallels=[pr.attrs["standard_parallel"]]
                )
            crs = CRS.from_wkt(projection.to_wkt())
        
            # Transform from lat-lon (ETEX data) to rotated pole coordinate system (harmonie data)
            transformer = Transformer.from_crs("+proj=longlat", crs)

            new_lon, new_lat = transformer.transform(old_long, old_lat)

    else:
        raise ValueError("Projection not added to script")
    
    return new_lon, new_lat 

In [ ]:
rot_lon, rot_lat = convert_coords(snap_data, 12.11076, 57.26006)
print(rot_lon, rot_lat)

In [ ]:
print(meteo_data['x_wind_10m'].sel(x=rot_lon, y=rot_lat, method='nearest').x.values)
print(meteo_data['x_wind_10m'].sel(x=rot_lon, y=rot_lat, method='nearest').y.values)
print(np.where(meteo_data.x.values==-175084)[0][0])
print(np.where(meteo_data.y.values==-670017.9)[0][0])